#content > div.spot.section_keyword > div.home_section.active > div > div.keyword_carousel > div > div > div:nth-child(9) > div > div > ul > li:nth-child(1) > a > em

#content > div.spot.section_keyword > div.home_section.active > div > div.keyword_carousel > div > div > div:nth-child(10) > div > div > ul > li:nth-child(1) > a > span

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#url = 'https://datalab.naver.com/keyword/realtimeList.naver?where=main'
url = "https://datalab.naver.com/"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')


rank_list = soup.select("ul.rank_list em")
list_list = soup.select("ul.rank_list span")
import csv


ranks = [rank.get_text(strip=True) for rank in rank_list]
lists = [item.get_text(strip=True) for item in list_list]


with open("rank_data.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Rank", "List"])

    for r, l in zip(ranks, lists):
        writer.writerow([r, l])

print("CSV 저장 완료: rank_data.csv")


CSV 저장 완료: rank_data.csv


In [4]:
import pandas as pd

# 1. 데이터 불러오기
df = pd.read_csv('rank_data.csv')

# 2. 날짜 리스트(예시: 17~26일까지 10개)
dates = [f"2024-07-{str(day).zfill(2)}" for day in range(17, 29)]  # 17일~26일(10개)
# 만약 17~28일까지 12개면 range(17,29)

# 3. 날짜를 10개씩 반복해서 전체 데이터 수에 맞게 생성
# (len(df) = 100이면, dates[0] * 10, dates[1] * 10, ...)
date_col = []
for d in dates:
    date_col += [d] * 10

# 4. 데이터 행 수와 날짜 리스트가 정확히 일치해야 함!
df['날짜'] = date_col[:len(df)]
df['카테고리'] = '패션의류'

# 5. 컬럼 순서 정리 후 저장
df = df[['날짜', 'Rank','카테고리', 'List']]
df.to_csv('rank_data_10perday.csv', encoding='utf-8-sig', index=False)
print(df.head(20))

            날짜  Rank  카테고리         List
0   2024-07-17     1  패션의류          원피스
1   2024-07-17     2  패션의류         블라우스
2   2024-07-17     3  패션의류          티셔츠
3   2024-07-17     4  패션의류        여름원피스
4   2024-07-17     5  패션의류  써스데이아일랜드원피스
5   2024-07-17     6  패션의류        듀엘원피스
6   2024-07-17     7  패션의류      쉬즈미스원피스
7   2024-07-17     8  패션의류       지고트원피스
8   2024-07-17     9  패션의류       시슬리원피스
9   2024-07-17    10  패션의류       잇미샤원피스
10  2024-07-18     1  패션의류          원피스
11  2024-07-18     2  패션의류  써스데이아일랜드원피스
12  2024-07-18     3  패션의류         블라우스
13  2024-07-18     4  패션의류     써스데이아일랜드
14  2024-07-18     5  패션의류        여름원피스
15  2024-07-18     6  패션의류      써스데이원피스
16  2024-07-18     7  패션의류       시슬리원피스
17  2024-07-18     8  패션의류       리스트원피스
18  2024-07-18     9  패션의류          티셔츠
19  2024-07-18    10  패션의류       지고트원피스


-----------------------------------------------------------------------------

In [19]:
import sqlite3
import mysql.connector

In [20]:

# 1. DataLab 데이터 불러오기 (csv, 혹은 크롤링 결과 df)
df = pd.read_csv('rank_data_10perday.csv')  

In [21]:
sqlite_conn=sqlite3.connect('Datalab.db')
sqlite_cursor=sqlite_conn.cursor()

In [16]:
mysql_conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='naver_datalab',
    charset='utf8mb4'
)
mysql_cursor = mysql_conn.cursor()

In [11]:
# 테이블 생성
sqlite_cursor.execute('''
CREATE TABLE IF NOT EXISTS Datalab_ranking (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    날짜 TEXT,
    Rank TEXT,
    카테고리 TEXT,
    List TEXT
)
''')

In [71]:
# 테이블 삭제(있으면)
sqlite_cursor.execute("DROP TABLE IF EXISTS datalab_ranking")

In [ ]:
import pandas as pd
import sqlite3

# 1. CSV 데이터프레임 불러오기
df = pd.read_csv('rank_data_10perday.csv')
print(df.head())         # 데이터 확인
print(df.columns)        # 컬럼 확인

# 2. SQLite 연결
sqlite_conn = sqlite3.connect('Datalab.db')
sqlite_cursor = sqlite_conn.cursor()

# 3. 테이블 DROP & CREATE (컬럼명 대소문자까지 완전히 일치!)
sqlite_cursor.execute("DROP TABLE IF EXISTS datalab_ranking")
sqlite_cursor.execute('''
CREATE TABLE datalab_ranking (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    날짜 VARCHAR(20),
    카테고리 VARCHAR(50),
    Rank VARCHAR(10),
    List VARCHAR(255)
)
''')

# 4. 데이터 삽입
sqlite_cursor.executemany(
    "INSERT INTO datalab_ranking ( 날짜, 카테고리, Rank, List) VALUES (?, ?, ?, ?)",
    df[['날짜', '카테고리', 'Rank', 'List']].values.tolist()
)
sqlite_conn.commit()

# 5. 데이터 조회
sqlite_cursor.execute("SELECT * FROM datalab_ranking")
rows = sqlite_cursor.fetchall()
print("데이터 개수:", len(rows))
for r in rows[:10]:
    print(r)

sqlite_conn.close()


           날짜  Rank  카테고리         List
0  2024-07-17     1  패션의류          원피스
1  2024-07-17     2  패션의류         블라우스
2  2024-07-17     3  패션의류          티셔츠
3  2024-07-17     4  패션의류        여름원피스
4  2024-07-17     5  패션의류  써스데이아일랜드원피스
Index(['날짜', 'Rank', '카테고리', 'List'], dtype='object')


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Rank VARCHAR(10),
    List VARCHAR(255)
)' at line 5

In [25]:
# 6. MySQL 연결
import mysql.connector

mysql_conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='naver_datalab',
    charset='utf8mb4'
)
mysql_cursor = mysql_conn.cursor()

# 7. MySQL 테이블 생성 (없으면 생성)
mysql_cursor.execute('''
CREATE TABLE IF NOT EXISTS datalab_ranking (
    id INT AUTO_INCREMENT PRIMARY KEY,
    날짜 VARCHAR(20),
    카테고리 VARCHAR(50),
    `Rank` VARCHAR(10),
    List VARCHAR(255)
)
''')

# 8. 데이터 삽입
mysql_cursor.executemany(
    "INSERT INTO datalab_ranking (날짜, 카테고리, `Rank`, List) VALUES (%s, %s, %s, %s)",
    df[['날짜', '카테고리', 'Rank', 'List']].values.tolist()
)
mysql_conn.commit()

# 9. 확인용 조회
mysql_cursor.execute("SELECT COUNT(*) FROM datalab_ranking")
print("MySQL 데이터 개수:", mysql_cursor.fetchone()[0])

mysql_conn.close()

MySQL 데이터 개수: 110


In [77]:
print(df.columns)

Index(['날짜', 'Rank', '카테고리', 'List'], dtype='object')


In [78]:
sqlite_cursor.execute("PRAGMA table_info(datalab_ranking)")
print(sqlite_cursor.fetchall())

[(0, 'id', 'INTEGER', 0, None, 1), (1, '날짜', 'VARCHAR(20)', 0, None, 0), (2, '카테고리', 'VARCHAR(50)', 0, None, 0), (3, '랭크', 'VARCHAR(10)', 0, None, 0), (4, '키워드', 'VARCHAR(255)', 0, None, 0)]


In [79]:
print(df[['날짜', '카테고리', 'Rank', 'List']].head())

           날짜  카테고리  Rank   List
0  2024-07-17  패션의류     1    원피스
1  2024-07-17  패션의류     2     우비
2  2024-07-17  패션의류     3   블라우스
3  2024-07-17  패션의류     4    티셔츠
4  2024-07-17  패션의류     5  여름원피스
